In [1]:
import sys
import os
sys.path.append(os.path.abspath('../input/sentence-transformers'))

In [2]:
from sentence_transformers import SentenceTransformer

/home/jaehoon_scatterlab_co_kr/projects/image-to-prompts/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
st_model = SentenceTransformer('../input/all-MiniLM-L6-v2')

In [10]:
out = st_model.encode(['hi','ja'])
out.shape

(2, 384)

In [112]:
import json
with open('../datasets/validation/metadata.jsonl') as f:
    data = [json.loads(d) for d in f.readlines()]
text_data = [d['text'] for d in data]

In [87]:
pairs[0]

(298, 7604)

In [88]:
text_data[298], text_data[7604]

('heisenberg, hyper detailed, digital art, ue, artstation, natural lighting, studio quality, smooth render, artgerm, greg rutkowski, peter mohrbacher, hajime sorayama, boris vallejo, craig mullins, mat drawinf paper',
 'epic westerner sangoku, hyper detailed, radiant, digital art, ue, artstation, natural lighting, studio quality, smooth render, artgerm, greg rutkowski, peter mohrbacher, hajime sorayama, boris vallejo, craig mullins, mat drawing paper')

In [114]:
embds = None
BS = 128
for i in tqdm(range(0, len(text_data), BS)):
    batch = text_data[i:i+BS]
    emb = torch.tensor(st_model.encode(batch))
    if embds is None:
        embds = emb
    else:
        embds = torch.vstack([embds, emb])

100%|██████████| 69/69 [00:03<00:00, 22.73it/s]


In [115]:
embds.shape

torch.Size([8798, 384])

In [116]:
similarities = torch.mm(embds, embds.T)

In [117]:
similarities.shape

torch.Size([8798, 8798])

In [118]:
idx = torch.nonzero(similarities > 0.85, as_tuple=True)

In [119]:
len(idx[0])

8832

In [121]:
pairs = []
for x,y in zip(idx[0].tolist(), idx[1].tolist()):
    if x == y:
        continue
    pairs.append((x,y))

In [122]:
len(pairs)

34

In [123]:
text_data[pairs[1][0]], text_data[pairs[1][1]]

('ominous forest, beautiful landscape, environment, colors, film, dramatic, cinematic, highly detailed, dark, large scale, hyperrealistic, realistic lighting, octane render, thomas kinkade, by wlop, artgerm, trending on artstation , clear, sharp',
 'beautiful environment, dream, colors, film, dramatic, highly detailed, mid day, large scale, hyperrealistic, realistic lighting, octane render, by wlop, artgerm, trending on artstation ')

In [124]:
delete_ids = [p[1] for p in pairs]
results = []
for idx, d in enumerate(data):
    if idx in delete_ids:
        continue
    else:
        results.append(d)

In [125]:
len(data), len(results)

(8798, 8765)

In [126]:
import json
with open('metadata_validation.jsonl', 'w') as f:
    for r in results:
        f.write(json.dumps(r))
        f.write('\n')

In [4]:
out = st_model.encode(["a guy opening a donut shop while the employees stare at him in disgust, digital art by julie bell and joe fenton. cgsociety contest winner, computer science fiction painterly aesthetic, darksynth, wiccan-mockup inspired, made of beads&knolling patterns on instagram, gotham city/diorama", "Kaggle employee Phil at a donut shop ordering all the best donuts, with a speech bubble that proclai"])

In [5]:
import torch
a = torch.tensor(out[0])
b = torch.tensor(out[1])
sim = torch.matmul(a,b.T)

/tmp/ipykernel_33044/134043059.py:4: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2981.)
  sim = torch.matmul(a,b.T)


In [6]:
sim

tensor(0.4744)